In [93]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import re

In [94]:
data = pd.read_json('../Parcer/Cleaning/data.json')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   product_id               133 non-null    object 
 1   type                     131 non-null    object 
 2   grace_period             131 non-null    object 
 3   credit_limit             131 non-null    object 
 4   first_year_fee           131 non-null    object 
 5   second_year_fee          122 non-null    object 
 6   min_age                  130 non-null    object 
 7   max_age                  130 non-null    object 
 8   citizenship              130 non-null    object 
 9   registration             130 non-null    object 
 10  work_experience          130 non-null    object 
 11  maximum                  131 non-null    object 
 12  repayment_method         131 non-null    object 
 13  max_monthly_cashback     131 non-null    object 
 14  ndfl_2_3_4               1

In [95]:
data = data.drop(columns=['product_name', 'bank', 'product_id'])

In [96]:
cat_col = ['bank_certificate', 'ndfl_2_3_4', 'other_documents', 'pension_certificate',
           'free_form_certificate', 'additional_income_proof', 'payment_system',
           'registration', 'citizenship']
for i in cat_col:
    print(data[i].value_counts(), end='\n\n')

bank_certificate
нет требуется                                                              91
по выбору, если требуется один или несколько документов из одной группы    35
обязательно                                                                 3
желательно или по требованию                                                2
Name: count, dtype: int64

ndfl_2_3_4
нет требуется                                                              61
по выбору, если требуется один или несколько документов из одной группы    44
обязательно                                                                20
желательно или по требованию                                                6
Name: count, dtype: int64

other_documents
нет требуется                                                              122
по выбору, если требуется один или несколько документов из одной группы      6
обязательно                                                                  2
желательно или по требованию             

In [97]:

onehotencoder = OneHotEncoder(sparse_output=False)
data_ohe = onehotencoder.fit_transform(data[cat_col])
one_hot_df = pd.DataFrame(data_ohe, columns=onehotencoder.get_feature_names_out(cat_col))
one_hot_df

,bank_certificate_желательно или по требованию,bank_certificate_нет требуется,bank_certificate_обязательно,"bank_certificate_по выбору, если требуется один или несколько документов из одной группы",bank_certificate_None,ndfl_2_3_4_желательно или по требованию,ndfl_2_3_4_нет требуется,ndfl_2_3_4_обязательно,"ndfl_2_3_4_по выбору, если требуется один или несколько документов из одной группы",ndfl_2_3_4_None,...,"registration_временная, постоянная","registration_временная, постоянная в любом регионе присутствия банка",registration_постоянная,registration_постоянная в любом регионе присутствия банка,registration_постоянная в регионе обращения,"registration_постоянная в регионе обращения, постоянная в любом регионе присутствия банка",registration_None,citizenship_не требуется,citizenship_требуется,citizenship_None
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
129,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
130,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
131,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [98]:
condition = (
    (one_hot_df['bank_certificate_None'] == 1.0) |
    (one_hot_df['ndfl_2_3_4_None'] == 1.0) |
    (one_hot_df['registration_None'] == 1.0) |
    (one_hot_df['citizenship_None'] == 1.0) |
    (one_hot_df['additional_income_proof_None'] == 1.0) | 
    (one_hot_df['pension_certificate_None'] == 1.0) |
    (one_hot_df['other_documents_None'] == 1.0)
)
one_hot_df = one_hot_df[~condition]
one_hot_df = one_hot_df.drop(columns=['payment_system_None', 'citizenship_None', 
                         'additional_income_proof_None', 'pension_certificate_None',
                         'other_documents_None', 'ndfl_2_3_4_None', 'bank_certificate_None',
                         'registration_None', 'free_form_certificate_None'])

In [99]:
data = data.drop(columns=cat_col)
data_encoded = pd.concat([data, one_hot_df], axis=1)
data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 55 columns):
 #   Column                                                                                           Non-Null Count  Dtype  
---  ------                                                                                           --------------  -----  
 0   type                                                                                             131 non-null    object 
 1   grace_period                                                                                     131 non-null    object 
 2   credit_limit                                                                                     131 non-null    object 
 3   first_year_fee                                                                                   131 non-null    object 
 4   second_year_fee                                                                                  122 non-null    object 
 5   min_age 

In [100]:
data_encoded['grace_period'].value_counts()

grace_period
55 дней        21
50 дней        16
62 дня         14
120 дней       13
100 дней        6
115 дней        5
110 дней        4
до 3 лет        4
180 дней        4
51 день         3
до 60 дней      3
60 дней         3
56 дней         3
Нет             2
150 дней        2
112 дней        2
3 года          2
130 дней        2
90 дней         2
до 62 дней      2
123 дня         1
1094 дня        1
40 дней         1
от 121 дня      1
61 день         1
до 56 дней      1
до 117 дней     1
755 дней        1
111 дней        1
до 115 дней     1
до 110 дней     1
415 дней        1
4 месяца        1
до 50 дней      1
212 дней        1
200 дней        1
2 месяца        1
30 дней         1
Name: count, dtype: int64

In [101]:
def convert_to_days(value):
    if isinstance(value, str):
        if "лет" in value or "года" in value:
            years = re.findall(r'\d+', value)
            if years:
                return float(years[0]) * 365
        
        elif "месяц" in value:
            months = re.findall(r'\d+', value)
            if months:
                return float(months[0]) * 30
        
        elif "до" in value:
            days = re.findall(r'\d+', value)
            if days:
                return float(days[0])
        
        elif "от" in value:
            days = re.findall(r'\d+', value)
            if days:
                return float(days[0])
        
        elif "день" in value or "дня" in value or "дней" in value:
            days = re.findall(r'\d+', value)
            if days:
                return float(days[0])
        
        else:
            return None
    
    return value


data_encoded['grace_period'] = data_encoded['grace_period'].apply(convert_to_days)

In [102]:
def split_and_clean(value):
    if pd.isna(value) or value in ['', ' ']:
        return []
    
    parts = re.split(r',', value)
    result = []
    
    for part in parts:

        if '/' in part:
            subparts = re.split(r'/', part)
            main_service = subparts[0].strip()
            for subpart in subparts:
                result.append(f"{main_service} {subpart.strip()}")
        else:
            result.append(part.strip())
    
    return result


data_encoded['type_split'] = data_encoded['type'].apply(split_and_clean)

all_categories = set()
for categories in data_encoded['type_split']:
    all_categories.update(categories)

for category in sorted(all_categories):
    data_encoded[category] = data_encoded['type_split'].apply(lambda x: 1 if category in x else 0)

data_encoded.drop(columns=['type_split', 'type'], inplace=True)


In [103]:
def parse_credit_limit(value):
    if pd.isna(value):
        return None, None
    
    value = value.replace("₽", "").replace(",", "").strip()
    
    lower_bound = None
    upper_bound = None
    
    if "до" in value and "от" not in value:
        match = re.search(r"до\s+(\d+)", value)
        if match:
            upper_bound = int(match.group(1).replace(" ", ""))
    
    elif "от" in value and "до" in value:
        match = re.search(r"от\s+(\d+)\s+до\s+(\d+)", value)
        if match:
            lower_bound = int(match.group(1).replace(" ", ""))
            upper_bound = int(match.group(2).replace(" ", ""))
    
    elif "от" in value and "до" not in value:
        match = re.search(r"от\s+(\d+)", value)
        if match:
            lower_bound = int(match.group(1).replace(" ", ""))
    
    elif value.isdigit():
        upper_bound = int(value.replace(" ", ""))
    
    elif "любой" in value:
        lower_bound = 0
        upper_bound = float("inf")
    
    return lower_bound, upper_bound


data_encoded[['below_b', 'upper_b']] = data_encoded['credit_limit'].apply(lambda x: pd.Series(parse_credit_limit(x)))
data_encoded.drop(columns='credit_limit', inplace=True)

In [104]:
data_encoded['min_age'] = data_encoded['min_age'].dropna().apply(lambda x: x[:2])
data_encoded['max_age'] = data_encoded['max_age'].dropna().apply(lambda x: x[:2])
data_encoded['min_age'] = data_encoded['min_age'].astype('float')
data_encoded['max_age'] = data_encoded['max_age'].astype('float')

In [105]:
categories = [
    'Такси', 'Кино и развлечения', 'Кафе и рестораны', 'Красота и здоровье', 
    'Одежда и обувь', 'Супермаркеты', 'Транспорт', 'Туры для путешествий', 
    'Бронирование отелей', 'Электроника и бытовая техника', 'Игры', 
    'Канцтовары и книги', 'Товары для животных', 'АЗС', 'Спорттовары', 
    'Дом и ремонт', 'Обучение', 'Каршеринг', 'Автоуслуги', 'Цветы', 
    'Аптеки', 'Детские товары', 'Авиабилеты', 'Ж/Д билеты', 'Фастфуд', 
    'Маркетплейсы', 'Театры', 'Оптика', 'Фитнес', 'Аренда автомобилей', 
    'Привет, Мир!', 'ЖКХ', 'На всё', 'У партнеров'
]

In [106]:
print('\n'.join(i for i in set(data_encoded['cashback'].values) if pd.isna(i) != True))

На всё: 1 %; Транспорт: 10 %; АЗС: 5 %; Такси: 5 %; Автоуслуги: 5 %; Каршеринг: 5 %; Аренда автомобилей: 3 %
У партнеров: 50 %
На всё: 1 %; Кафе и рестораны: 8 %; Кино и развлечения: 8 %; Аптеки: 8 %; Такси: 8 %; Дом и ремонт: 8 %; Авиабилеты: 8 %; Аренда автомобилей: 8 %; Бронирование отелей: 8 %; Транспорт: 8 %; Красота и здоровье: 8 %; Автоуслуги: 8 %; Фастфуд: 8 %; АЗС: 8 %; Спорттовары: 8 %; Каршеринг: 8 %; Цветы: 8 %; Детские товары: 8 %; Фитнес: 8 %; Одежда и обувь: 8 %
У партнеров: 35 %
У партнеров: 5 %; На всё: 1 %
У партнеров: 20 %; Канцтовары и книги: 25 %; ЖКХ: 10 %; Цветы: 5 %; Супермаркеты: 5 %; Красота и здоровье: 3 %; Аптеки: 3 %; Одежда и обувь: 3 %
Супермаркеты: 2 %; Ж/Д билеты: 2 %; Такси: 2 %; Кафе и рестораны: 2 %; Транспорт: 2 %
У партнеров: 30 %; Красота и здоровье: 10 %; Спорттовары: 10 %; Каршеринг: 10 %
Привет, Мир!: 20 %; У партнеров: 30 %
У партнеров: 25 %; На всё: 1 %; Красота и здоровье: 5 %; Ж/Д билеты: 5 %; Электроника и бытовая техника: 5 %; АЗС: 5 %; Т

In [107]:
temp_df = pd.DataFrame()
for index, row in data_encoded[['cashback']].iterrows():
    if row['cashback'] != None:
        trow = ''.join(i for i in row.values).split(';')
        for i in trow:
            i = i.split(':')
            i1 = i[1]
            temp_df.loc[index, i[0].lstrip(' ')] = i1[1:len(i1)-2]
for i in temp_df.columns:
    temp_df[i] = temp_df[i].astype('str')
    temp_df[i] = temp_df[i].str.replace(',', '.')
    temp_df[i] = temp_df[i].astype('float')
    temp_df[i] = temp_df[i].fillna(0)
data = pd.concat([data_encoded, temp_df], axis=1)

In [108]:
print(data['mmtc_in'].values)

['0% (до 500000)' ' ' ' ' '0% (до 60000)' None ' ' '0% (Любая)'
 '0% (Любая)' ' ' '0% (до 150000)' '5,9% (Любая)' '0% (Любая)'
 '0% (Любая)' '0% (Любая)' ' ' '0% (Любая)' '0% (до 150000)' ' ' ' '
 '0% (до 50000)' '0% (до 300000)' '0% (Любая)' '0% (Любая)'
 '0% (до 20000)' '0% (Любая)' '0% (до 50000)' '0% (до 100000)' ' ' ' ' ' '
 ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '3,5% (Любая)' '3,9% (Любая)' ' '
 '0% (до 100000)' '3% (Любая)' '0% (Любая)' '0% (Любая)' ' ' ' '
 '0% (до 50000)' '0% (Любая)' ' ' '0% (Любая)' ' ' ' ' ' ' '3,9% (Любая)'
 ' ' ' ' '4% (Любая)' '0% (до 150000)' '0% (Любая)' '0% (до 1000000)' ' '
 '0% (Любая)' '0% (Любая)' ' ' '0% (Любая)' '0% (Любая)' '0% (Любая)' ' '
 '0% (Любая)' ' ' ' ' ' ' ' ' ' ' '0% (Любая)' ' ' '0% (Любая)'
 '0% (до 30000)' ' ' ' ' '4% (Любая)' '0% (до 150000)' '0% (Любая)' ' '
 ' ' None '1% (Любая)' ' ' '0% (Любая)' '0% (Любая)' ' ' ' ' ' '
 '2,5% (Любая)' ' ' '4,5% (Любая)' ' ' ' ' '3% (Любая)' ' ' ' '
 '0% (Любая)' '0% (до 50000)' ' ' ' ' '4% (Любая)'

In [109]:
def parse_to_float(value):
    """
    Функция для преобразования строки в float.
    Удаляет текстовые части, такие как "в день" и "в месяц", и преобразует число в float.
    """
    if value is None or value.strip() == '':
        return None
    
    cleaned_value = re.sub(r'[^\d.,]+', '', value.replace(' ', ''))
    
    cleaned_value = cleaned_value.replace(',', '.')
    
    try:
        return float(cleaned_value)
    except ValueError:
        return None


b_ = ['mmtc_b_in', 'mmtc_b_out', 'cmtc_b_in', 'cmtc_b_out']



for i in b_:
    parsed_data = [parse_to_float(value) for value in data[i]]
    data[i] = parsed_data
    data[i].astype('float').fillna(0.0)

In [110]:

def parse_percentage(value):
    if pd.isna(value) or value.strip() == '':
        return None
    match = re.search(r'([\d.,]+)%', str(value))
    if match:
        percentage = match.group(1).replace(',', '.')
        try:
            return float(percentage)
        except ValueError:
            return None
    return None

def parse_amount_in_parentheses(value):
    if pd.isna(value) or value.strip() == '':
        return None
    match = re.search(r'\((.*?)\)', str(value))
    if match:
        amount_str = match.group(1).strip()
        if amount_str.lower() == 'любая':
            return None
        number_match = re.search(r'([\d.,]+)', amount_str)
        if number_match:
            amount = number_match.group(1).replace(' ', '').replace(',', '.')
            try:
                return float(amount)
            except ValueError:
                return None
    return None


tc_ = ['cmtc_in', 'cmtc_out', 'mmtc_in', 'mmtc_out']

for col in tc_:
    # Парсим проценты и суммы
    percentages = [parse_percentage(val) for val in data[col]]
    amounts = [parse_amount_in_parentheses(val) for val in data[col]]
    
    # Преобразуем в DataFrame (если data - словарь списков)
    data[f'{col}_pr'] = pd.Series(percentages).astype('float').fillna(0.0)
    data[f'{col}_am'] = pd.Series(amounts).astype('float').fillna(0.0)

# Результат
print("cmtc_in_pr:", data['cmtc_in_pr'].values[0])
print("cmtc_in_am:", data['cmtc_in_am'].values[0])
print("cmtc_out_pr:", data['cmtc_out_pr'].values[0])
print("cmtc_out_am:", data['cmtc_out_am'].values[0])

cmtc_in_pr: 2.9
cmtc_in_am: 0.0
cmtc_out_pr: 2.9
cmtc_out_am: 0.0


In [111]:
data.to_json('sample.json', orient='records', force_ascii=False, indent=2)

In [112]:
print(data['max_monthly_cashback'])
print(data['repayment_method'])
print(data['maximum'])
print(data['work_experience'])
print(data['first_year_fee'])
print(data['second_year_fee'])


0                    
1                    
2                    
3                    
4                None
            ...      
128     5 000 бонусов
129    17 000 бонусов
130                  
131      3 000 баллов
132             5 000
Name: max_monthly_cashback, Length: 133, dtype: object
0      валюта карты
1      валюта карты
2      валюта карты
3      валюта карты
4              None
           ...     
128          бонусы
129          бонусы
130    валюта карты
131           баллы
132    валюта карты
Name: repayment_method, Length: 133, dtype: object
0      : до 30 %
1      : до 30 %
2           до 0
3      : до 35 %
4           None
         ...    
128    : до 25 %
129    : до 30 %
130         до 0
131     : до 3 %
132    : до 33 %
Name: maximum, Length: 133, dtype: object
0                                     не менее 3 месяцев
1                                     не менее 3 месяцев
2                                     не менее 6 месяцев
3      Для физических лиц: не ме

In [113]:
def parse_max_monthly_cashback(value):
    if pd.isna(value):
        return None
    value = str(value)
    cleaned_value = re.sub(r"[^\d\s]", "", value)
    cleaned_value = cleaned_value.replace(" ", "")
    if cleaned_value == "":
        return None
    return float(cleaned_value)

data["max_monthly_cashback"] = data["max_monthly_cashback"].apply(parse_max_monthly_cashback)
data["max_monthly_cashback"] = data["max_monthly_cashback"].astype('float').fillna(0.0)
def parse_maximum(value):
    if pd.isna(value):
        return None
    value = str(value)
    match = re.search(r"\d+", value)
    if match:
        return float(match.group())
    return None

data["maximum"] = data["maximum"].apply(parse_maximum)
data["maximum"] = data["maximum"].astype('float').fillna(0.0)

In [114]:
print(data["max_monthly_cashback"])

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
        ...   
128     5000.0
129    17000.0
130        0.0
131     3000.0
132     5000.0
Name: max_monthly_cashback, Length: 133, dtype: float64


In [115]:
labelencoder = LabelEncoder()
data['repayment_method'] = labelencoder.fit_transform(data['repayment_method'].values)

In [116]:
data.drop(columns=['work_experience', 'cashback', 'second_year_fee', 
                   'first_year_fee', 'first_year_fee', 'cmtc_in', 
                   'cmtc_b_in', 'cmtc_b_out', 'mmtc_in', 'mmtc_out',
                   'mmtc_b_in', 'mmtc_b_out', 'cmtc_out'], inplace=True)

In [117]:
data.to_json('sample.json', orient='records', force_ascii=False, indent=2)